<a href="https://colab.research.google.com/github/romiebanerjee/sae-tools/blob/master/notebooks/SAE_interpretation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install transformer_lens

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import blobfile as bf

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device: ", device)

Using device:  cpu


In [3]:
!git clone https://github.com/romiebanerjee/sae-tools.git
%cd sae-tools
import sparse_autoencoder as sae

fatal: destination path 'sae-tools' already exists and is not an empty directory.
/content/sae-tools


In [4]:
import transformer_lens

In [5]:
model = transformer_lens.HookedTransformer.from_pretrained("gpt2", center_writing_weights=False).to(device)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Loaded pretrained model gpt2 into HookedTransformer
Moving model to device:  cpu


In [6]:
def get_layer_name(location: str, layer_index:int):
    transformer_lens_loc = {
        "mlp_post_act": f"blocks.{layer_index}.mlp.hook_post",
        "resid_delta_attn": f"blocks.{layer_index}.hook_attn_out",
        "resid_post_attn": f"blocks.{layer_index}.hook_resid_mid",
        "resid_delta_mlp": f"blocks.{layer_index}.hook_mlp_out",
        "resid_post_mlp": f"blocks.{layer_index}.hook_resid_post",
    }[location]

    return transformer_lens_loc

In [7]:
## download gpt-2 SAE weights from Openai
layer_index = 10
location = "resid_post_mlp"

with bf.BlobFile(sae.paths.v5_128k(location, layer_index), mode="rb") as f:
    print(f"fetching weights for sae at {get_layer_name(location, layer_index)} from {sae.paths.v5_128k(location, layer_index)}")
    state_dict = torch.load(f)

fetching weights for sae at blocks.10.hook_resid_post from az://openaipublic/sparse-autoencoder/gpt2-small/resid_post_mlp_v5_128k/autoencoders/10.pt


In [8]:
import requests

# Download Frankenstein from Project Gutenberg
url = "https://www.gutenberg.org/files/84/84-0.txt"  # Frankenstein's permanent URL
response = requests.get(url)

# Save to a file
with open("frankenstein.txt", "w", encoding="utf-8") as f:
    f.write(response.text)

In [9]:
# Load the entire file into a string
with open('frankenstein.txt', 'r', encoding='utf-8') as file:
    text = file.read()  # Entire content as a single string
#split into smaller strings
import re
sentences = re.split(r'(?<=[.!?\n])\s+', text)
print(len(sentences))

3446


In [120]:
# model.to_tokens(sentences)
batch_tokens = model.tokenizer(sentences, padding=True, return_tensors="pt")['input_ids']
batch_masks = model.tokenizer(sentences, padding=True, return_tensors="pt")['attention_mask']
print(batch_tokens.shape)
print(batch_masks.shape)

def random_row_indices(tensor, r):
    R = tensor.size(0)
    # Generate random indices without replacement
    rand_indices = torch.randperm(R)[:r]
    return rand_indices

rand_indices = random_row_indices(batch_tokens, 100)

batch_tokens = batch_tokens[rand_indices]
batch_masks = batch_masks[rand_indices]

print(batch_tokens.shape)
print(batch_masks.shape)


torch.Size([3446, 212])
torch.Size([3446, 212])
torch.Size([100, 212])
torch.Size([100, 212])


In [153]:
for i,r in enumerate(rand_indices):
  print(i, sentences[r])

0 He has already had one or two little _wives,_ but Louisa Biron
is his favourite, a pretty little girl of five years of age.
1 My lieutenant, for instance, is a man of wonderful courage
and enterprise; he is madly desirous of glory, or rather, to word my phrase
more characteristically, of advancement in his profession.
2 “I thank you, Walton,” he said, “for your kind intentions towards so
miserable a wretch; but when you speak of new ties and fresh
affections, think you that any can replace those who are gone?
3 We left Oxford with regret and proceeded to Matlock, which was our next
place of rest.
4 The past appeared to me in the light of a frightful dream;
yet the vessel in which I was, the wind that blew me from the detested
shore of Ireland, and the sea which surrounded me, told me too forcibly
that I was deceived by no vision and that Clerval, my friend and dearest
companion, had fallen a victim to me and the monster of my creation.
5 “‘I am about to undertake that task; and it is

In [165]:
layer_name = get_layer_name(location, layer_index)
with torch.no_grad():
  batch_logits, batch_activation_cache = model.run_with_cache(batch_tokens, remove_batch_dim=False)
print(batch_logits.shape)
print(batch_activation_cache[layer_name].shape)

torch.Size([100, 212, 50257])
torch.Size([100, 212, 768])


In [172]:
input = "The rain pattered dismally against the panes, and my candle was out"
str_tokens = model.to_str_tokens(input)
print(str_tokens)
tokens = model.to_tokens(input)
print(tokens)
with torch.no_grad():
  logits, activation_cache = model.run_with_cache(input, remove_batch_dim=True)
print(logits.shape)
print(activation_cache[layer_name].shape)

['<|endoftext|>', 'The', ' rain', ' pat', 'tered', ' dism', 'ally', ' against', ' the', ' pan', 'es', ',', ' and', ' my', ' candle', ' was', ' out']
tensor([[50256,   464,  6290,  1458,  4400, 18099,   453,  1028,   262,  3425,
           274,    11,   290,   616, 26839,   373,   503]])
torch.Size([1, 17, 50257])
torch.Size([17, 768])


In [183]:
token_index = -1

layer_name = get_layer_name(location, layer_index)

act = activation_cache[layer_name][token_index, :]

input_tensor = activation_cache[layer_name]

autoencoder = sae.Autoencoder.from_state_dict(state_dict).to(device)

with torch.no_grad():
    latent_activations, info = autoencoder.encode(input_tensor)
    reconstructed_activations = autoencoder.decode(latent_activations, info)


decoder_weights = autoencoder.decoder.weight
act_dots = act @ decoder_weights
top_feature_indx = torch.argmax(act_dots)
top_feature = decoder_weights[:, top_feature_indx]

print(f'input tensor: {input_tensor.shape}')
print(f'latent activations: {latent_activations.shape}')
print(f'reconstructed activations: {reconstructed_activations.shape}')
print(f'act_dots: {act_dots.shape}')
print(f'top_feature index: {top_feature_indx}')
print(f'top_feature: {top_feature.shape}')

# print(torch.nonzero(latent_activations[50, :]))

input tensor: torch.Size([17, 768])
latent activations: torch.Size([17, 131072])
reconstructed activations: torch.Size([17, 768])
act_dots: torch.Size([131072])
top_feature index: 44733
top_feature: torch.Size([768])


In [186]:
act @ top_feature

tensor(70.7726, grad_fn=<DotBackward0>)

In [185]:
batch_acts = batch_activation_cache[layer_name].detach().cpu()
print(batch_acts.shape)

torch.Size([100, 212, 768])


In [187]:
batch_acts_top = batch_acts @ top_feature
mask = batch_acts_top > 0.
batch_acts_top = batch_acts_top * mask
print(batch_acts_top.shape)

torch.Size([100, 212])


In [188]:
for line, i in enumerate(range(100)):
  print(line, list(zip(model.to_str_tokens(sentences[rand_indices[i]]), batch_acts_top[i, :].data)))
  print('---------------------------------------------------------------------------------------------------------------')

0 [('<|endoftext|>', tensor(-0.)), ('He', tensor(-0.)), (' has', tensor(-0.)), (' already', tensor(-0.)), (' had', tensor(-0.)), (' one', tensor(-0.)), (' or', tensor(-0.)), (' two', tensor(-0.)), (' little', tensor(-0.)), (' _', tensor(-0.)), ('wives', tensor(-0.)), (',', tensor(-0.)), ('_', tensor(-0.)), (' but', tensor(-0.)), (' Louis', tensor(-0.)), ('a', tensor(-0.)), (' B', tensor(-0.)), ('iron', tensor(-0.)), ('\n', tensor(-0.)), ('is', tensor(-0.)), (' his', tensor(-0.)), (' favourite', tensor(-0.)), (',', tensor(-0.)), (' a', tensor(-0.)), (' pretty', tensor(-0.)), (' little', tensor(-0.)), (' girl', tensor(-0.)), (' of', tensor(-0.)), (' five', tensor(1.6644)), (' years', tensor(-0.)), (' of', tensor(-0.)), (' age', tensor(-0.)), ('.', tensor(-0.))]
---------------------------------------------------------------------------------------------------------------
1 [('<|endoftext|>', tensor(-0.)), ('My', tensor(-0.)), (' lieutenant', tensor(-0.)), (',', tensor(-0.)), (' for', ten

In [190]:
max, indx = torch.max(batch_acts_top, dim=1)

In [191]:

torch.max(max), sentences[rand_indices[torch.argmax(max)]]

(tensor(59.0098, grad_fn=<MaxBackward1>),
 '“The next morning Felix went out to his work, and after the usual\noccupations of Agatha were finished, the Arabian sat at the feet of the\nold man, and taking his guitar, played some airs so entrancingly\nbeautiful that they at once drew tears of sorrow and delight from my\neyes.')

In [192]:
# acts_sentences = torch.sum(batch_acts_top, dim = 1)

for i in range(100):
  print(i, list(zip((max[i].data, sentences[rand_indices[i]]))))


0 [(tensor(1.6644),), ('He has already had one or two little _wives,_ but Louisa Biron\nis his favourite, a pretty little girl of five years of age.',)]
1 [(tensor(5.0871),), ('My lieutenant, for instance, is a man of wonderful courage\nand enterprise; he is madly desirous of glory, or rather, to word my phrase\nmore characteristically, of advancement in his profession.',)]
2 [(tensor(13.8025),), ('“I thank you, Walton,” he said, “for your kind intentions towards so\nmiserable a wretch; but when you speak of new ties and fresh\naffections, think you that any can replace those who are gone?',)]
3 [(tensor(1.3640),), ('We left Oxford with regret and proceeded to Matlock, which was our next\nplace of rest.',)]
4 [(tensor(8.4295),), ('The past appeared to me in the light of a frightful dream;\nyet the vessel in which I was, the wind that blew me from the detested\nshore of Ireland, and the sea which surrounded me, told me too forcibly\nthat I was deceived by no vision and that Clerval, my 

In [193]:
from IPython.display import HTML, display

def highlight_text(words, values, cmap='RdYlGn'):
    """
    Highlight words in text with color gradient based on values.
    Perfect for displaying in Colab notebooks.
    """
    # Normalize values to [0, 1] if not already
    values = np.array(values, dtype=float)
    if values.min() < 0 or values.max() > 1:
        values = (values - values.min()) / (values.max() - values.min() + 1e-8)

    # Get colormap
    cmap = plt.get_cmap(cmap)

    # Create highlighted HTML
    span = ""
    for word, val in zip(words, values):
        # Get RGBA color from colormap
        rgba = cmap(val)
        # Convert to hex
        color = '#%02x%02x%02x' % (int(rgba[0]*255), int(rgba[1]*255), int(rgba[2]*255))
        # Create span with background color
        span += f'<span style="background-color: {color}; border-radius: 3px; padding: 0 2px;">{word}</span>'
    return span


HTML(highlight_text(model.to_str_tokens(input), activation_cache[layer_name]@top_feature.detach() ))

In [194]:

for line, i in enumerate(range(100)):
  # print(list(zip(model.to_str_tokens(sentences[rand_indices[i]]), batch_acts_top[i, :].data)))
  # text = sentences[rand_indices[i]]
  words = model.to_str_tokens(sentences[rand_indices[i]])
  values = batch_acts_top[i, :].data

  display(HTML(f'<div style="font-size: 16px; line-height: 1.6; padding: 10px;">{highlight_text(words, values)}</div>'))
  print('------------------------------')

------------------------------


------------------------------


------------------------------


------------------------------


------------------------------


...

------------------------------


------------------------------


------------------------------


------------------------------


------------------------------
